In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from time import localtime, strftime
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [ ]:
def get_web(item_url):
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"}
    response = requests.get(item_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [ ]:
def send_mail(tittle, item_url, base_price, discount_perc):
    mailfrom = 'ng.tridung1610@gmail.com'
    pwd = input('Please type your sender password:')
    mailto = 'ng.tridung97@gmail.com'
    subject = tittle + ' from your Tiki wishlist is now on sale!'

    msg = MIMEMultipart('alternative')
    msg['Subject'] = subject
    msg['From'] = mailfrom
    msg['To'] = mailto

    body = '1 item you have wished for is now on sale!: ' + item_url  + '\n\n' \
        + '\t' + 'Item: ' + tittle + '.' + '\n' \
        + '\t' + 'Price: ' + '{:,.2f}₫'.format(float(base_price)) + '.' + '\n' \
        + '\t' + 'Discount: ' + str(discount_perc*-1) + '%.' + '\n\n' \
        + 'Specific pricing and discounts may be subject to change. Please check above link for details.' + '\n\n' \
        + 'Best Regards,'

    body = MIMEText(body, 'plain')
    msg.attach(body)
    msg = msg.as_string().encode('utf8')

    try:
        current_time = strftime('%d-%m-%Y %H:%M:%S', localtime())
        context = ssl.create_default_context()
        server = smtplib.SMTP('smtp.gmail.com', 587)
        with server:
            server.ehlo()
            server.starttls(context=context)
            server.ehlo()
            server.login(mailfrom, pwd)
            server.sendmail(mailfrom, mailto, msg)
            server.quit()
        print('Item: ' + tittle + ' | Email sent at: ' + current_time)
    except Exception as err:
        print(err)

In [ ]:
def check_price():
    item_url = 'https://tiki.vn/nhung-ke-xuat-chung-tai-ban-2019-p13332055.html?spid=13332056'
    soup = get_web(item_url)

    try:
        tittle = soup.find(lambda tag: tag.name == 'h1' and tag.get('class') == ['title']).get_text().strip()
        ori_price = soup.find(lambda tag: tag.name == 'p' and tag.get('class') == ['original-price']).get_text().replace('.', '').replace(' ₫', '').replace(' ', '').replace('Giáthịtrường:', '').strip()
        base_price = soup.find(lambda tag: tag.name == 'p' and tag.get('class') == ['price']).get_text().replace('.', '').replace(' ₫', '').replace(' ', '').strip()
        discount_perc = int((float(ori_price) - float(base_price))*100/float(ori_price))
        vendor = soup.find(lambda tag: tag.name == 'a' and tag.get('class') == ['seller-name']).get_text().strip()
        
        if(discount_perc >= 15): #more than 15% discount
            send_mail(tittle, item_url, base_price, discount_perc)
    except Exception as err:
        print(err)

In [ ]:
while(True):
    check_price()
    time.sleep(300)
    
# Currently I'm thinking about get historical pricing for a week. Then make a function to compare curent price with the average last week. Crawl data can be store in DB or csv.